# 大语言模型预训练-微调技术之LoRA

## 如何注入LoRA

In [2]:
from torch import nn
from peft import LoraConfig,get_peft_model

# 定义一个简单的3层的神经网络
net1=nn.Sequential(
       nn.Linear(10,20),
       nn.ReLU(),
       nn.Linear(20,2)
)
net1

Sequential(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): ReLU()
  (2): Linear(in_features=20, out_features=2, bias=True)
)

In [4]:
# 在第1层增加旁路
config=LoraConfig(target_modules=["0"])
net2=get_peft_model(net1,config)
net2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=20, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=20, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): Linear(in_features=20, out_features=2, bias=True)
    )
  )
)

## LoRA代码实践

### 步骤1 导入相关包

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import os

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'

### 步骤2 加载数据集

In [ ]:
ds = load_dataset("llm-wizard/alpaca-gpt4-data-zh")
ds

In [ ]:
ds[:1]

### 步骤3 数据集预处理

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

In [ ]:
def process_func(example):
    # 设置最大长度为256
    MAX_LENGTH = 256
    # 初始化输入ID、注意力掩码和标签列表
    input_ids, attention_mask, labels = [], [], []
    # 对指令和输入进行编码
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    # 对输出进行编码，并添加结束符
    response = tokenizer(example["output"] + tokenizer.eos_token)
    # 将指令和响应的输入ID拼接起来
    input_ids = instruction["input_ids"] + response["input_ids"]
    # 将指令和响应的注意力掩码拼接起来
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    # 将指令的标签设置为-100，表示不计算损失；将响应的输入ID作为标签
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    # 如果输入ID的长度超过最大长度，截断输入ID、注意力掩码和标签
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    # 返回处理后的数据
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

### 步骤4 创建模型

In [ ]:
#这行代码从Hugging Face Model Hub加载了一个预训练的Bloom模型，
# 模型名称为"Langboat/bloom-1b4-zh"，并且设置了low_cpu_mem_usage=True以减少CPU内存使用。
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)

In [ ]:
# 查看总共有哪些层，可以基于这些层添加LoRA
for name, parameter in model.named_parameters():
    print(name)

#### 1、PEFT 步骤1 配置文件

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
config = LoraConfig(task_type=TaskType.CAUSAL_LM)
##也可以不使用默认的，自己指定， 目标层 target_modules=["query_key_value"],秩 r=8
#config = LoraConfig(task_type=TaskType.CAUSAL_LM,r=8, target_modules=['query_key_value','dense_4h_to_h'])
config

#### 2、PEFT 步骤2 创建模型

In [ ]:
# 使用PEFT和预训练模型来创建一个微调模型。
# 这个模型将包含原始的预训练模型以及由PEFT引入的低秩参数
model = get_peft_model(model, config)
model

In [ ]:
# 查看Lora配置
config

### 步骤5 配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/cache/finetuning/bloom-1b4-zh-lora",# 指定模型训练结果的输出目录。
    per_device_train_batch_size=4, # 指定每个设备（如GPU）上的批次大小
    gradient_accumulation_steps=8,# 指定梯度累积步数。在本例子中，每8个步骤进行一次梯度更新。
    logging_steps=20, #指定日志记录的频率。在本例子中，每20个步骤记录一次日志
    num_train_epochs=4 #指定训练的总轮数
)

### 步骤6 创建训练器

In [ ]:
trainer = Trainer(
    model=model,#指定训练模型
    args=args, #指定训练参数
    train_dataset=tokenized_ds, #指定数据集
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True) #指定数据收集器。其中tokenizer是分词器，padding=True表示对输入进行填充以保持批次大小一致。
)

### 步骤7 模型训练

In [ ]:
trainer.train()

### 步骤8 模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

ipt = "Human: {}\n{}".format("如何写好一个简历？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )

## 主路合并旁路

### 1、加载基础模型

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

from peft import PeftModel

model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")

### 2、加载LoRA模型

In [ ]:
p_model = PeftModel.from_pretrained(model, model_id="/root/autodl-tmp/cache/finetuning/bloom-1b4-zh-lora/checkpoint-500")
p_model

### 3、模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=p_model, tokenizer=tokenizer, device=0)
ipt = "Human: {}\n{}".format("如何写好一个简历？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )

### 4、模型合并

In [ ]:
merge_model = p_model.merge_and_unload()
merge_model

### 5、模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=merge_model, tokenizer=tokenizer, device=0)
ipt = "Human:如何写好一个简历？\n\nAssistant: "
pipe(ipt, max_length=256,)

### 6、完整模型保存

In [ ]:
merge_model.save_pretrained("/root/autodl-tmp/cache/finetuning/bloom-1b4-zh-lora-merge")